In [ ]:
import numpy as np
import math
import cv2
import os

from scipy.signal import convolve2d

os.makedirs('outputs', exist_ok=True)
os.makedirs('outputs/part1', exist_ok=True)

def build_kernel(r):
    kernel = np.zeros((2*r+1,2*r+1))
    for i in range(2*r+1):
        for j in range(2*r+1):
            dis = math.sqrt((i-r)**2 + (j-r)**2)
            if(dis<=r-0.5):
                kernel[i][j] = 1
            elif(r-0.5 < dis < r+0.5):
                kernel[i][j] = r + 0.5 - dis
    kernel_disp = np.uint8(cv2.normalize(kernel, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part1/kernel_{r}.png',kernel_disp)
    return kernel

def convolve(image, r):
    kernel = build_kernel(r)
    output = convolve2d(image, kernel, mode = 'same', boundary='symm')
    output_disp = np.uint8(cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part1/spatial_blurring_{r}.png',output_disp)
    return output


img1 = cv2.imread('temple.png', cv2.IMREAD_GRAYSCALE)

img2 = convolve(img1, 8)

In [ ]:
def pad_image(img):
    m, n = img.shape[:2]
    return cv2.copyMakeBorder(img, 0, m, 0, n, cv2.BORDER_REFLECT)

def pad_kernel(kernel, m, n):
    a,b = kernel.shape[:2]
    padded_kernel = np.zeros((2*m,2*n))
    padded_kernel[:a,:b] = kernel
    return padded_kernel

def apply_freq_domain_blurring(image, r):
    m,n = image.shape[:2]
    kernel = build_kernel(r)
    kernel = pad_kernel(kernel, m ,n)
    padded_image = pad_image(image)
    cv2.imwrite(f'outputs/part1/padded_image.png',padded_image)
    kernel_disp = np.uint8(cv2.normalize(kernel, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part1/padded_kernel.png', kernel_disp )

    freq_img = np.fft.fft2(padded_image)
    ps_img_log = np.log(np.abs(freq_img)+1e-6)
    ps_img_disp = np.uint8(cv2.normalize(ps_img_log, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part1/power_spectrum_image.png',ps_img_disp)
    cv2.imwrite(f'outputs/part1/power_spectrum_image_shifted.png',np.fft.fftshift(ps_img_disp))

    freq_kernel = np.fft.fft2(kernel)
    ps_kernel_log = np.log(np.abs(freq_kernel)+1e-6)
    ps_kernel_disp = np.uint8(cv2.normalize(ps_kernel_log, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part1/power_spectrum_kernel_{r}.png',ps_kernel_disp)
    cv2.imwrite(f'outputs/part1/power_spectrum_kernel_shifted_{r}.png',np.fft.fftshift(ps_kernel_disp))

    freq_blurred_img = freq_img * freq_kernel
    freq_blurred_img_log = np.log(np.abs(freq_blurred_img)+ 1e-6)
    freq_blurred_img_disp = np.uint8(cv2.normalize(freq_blurred_img_log, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part1/power_spectrum_blurred_image_{r}.png',freq_blurred_img_disp)
    cv2.imwrite(f'outputs/part1/power_spectrum_blurred_image_shifted_{r}.png',np.fft.fftshift(freq_blurred_img_disp))

    blurred_img = np.fft.ifft2(freq_blurred_img).real
    blurred_img_disp = np.uint8(cv2.normalize(blurred_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part1/freq_blurring_padded_image_{r}.png',blurred_img_disp)
    blurred_img = blurred_img[0:m,0:n]
    blurred_img_disp = np.uint8(cv2.normalize(blurred_img, None, 0, 255, cv2.NORM_MINMAX))
    cv2.imwrite(f'outputs/part1/freq_blurring_{r}.png',blurred_img_disp)
    return blurred_img

apply_freq_domain_blurring(img1, 8)

In [ ]:
import time
import matplotlib.pyplot as plt

m,n = img1.shape[:2]
spatial_execution_time = []
r  = []
a = 1
t = 0
while(t<= 60 and a <= m/2 and a<= n/2):
    start = time.time()
    convolve(img1,a)
    end = time.time()
    execution_time = end - start
    spatial_execution_time.append(execution_time)
    r.append(a)
    a*=2
    t = execution_time
    print(f"Execution time: {execution_time} seconds")


freq_execution_time = []
r  = []
a = 1
t = 0
while(t<= 60 and a <= m/2 and a<= n/2):
    start = time.time()
    apply_freq_domain_blurring(img1,a)
    end = time.time()
    execution_time = end - start
    freq_execution_time.append(execution_time)
    r.append(a)
    a*=2
    t = execution_time
    print(f"Execution time: {execution_time} seconds")

plt.figure(figsize=(8, 6))


plt.plot(r, spatial_execution_time, label="Spatial Blurring", marker='o')


plt.plot(r, freq_execution_time, label="Frequency Blurring", marker='x')


plt.xscale('log', base=2)
plt.yscale('log', base=10)
plt.xlabel('Kernel Size (log2 scale)')
plt.ylabel('Execution Time (log10 scale) seconds')
plt.title('Comparison of Spatial vs Frequency Blurring Execution Time')
plt.legend()
plt.grid(True, which="both", ls="--")


plt.show()
